In [3]:
%pip install --upgrade --quiet  text-generation transformers google-search-results numexpr langchainhub sentencepiece jinja2

Note: you may need to restart the kernel to use updated packages.


In [1]:
import dotenv 
import os

dotenv.load_dotenv()

os.environ["DEBUG"] = "1"  # Set to "1" if you want to use debug mode.
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")

from huggingface_hub.hf_api import HfFolder
HfFolder.save_token(os.environ["HUGGINGFACEHUB_API_TOKEN"])


In [2]:
from typing import List
from typing import Tuple

from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.chat_models.huggingface import ChatHuggingFace

In [3]:
#repo_id = "mistralai/Mistral-7B-Instruct-v0.2"
# repo_id="HuggingFaceH4/zephyr-7b-beta"
#repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1"
#repo_id="microsoft/phi-2"
repo_id="meta-llama/Llama-2-70b-chat-hf"

llm = HuggingFaceEndpoint(
    repo_id=repo_id, 
    temperature=0.3, 
    max_new_tokens=512,
    top_k=30,
    repetition_penalty=1.03
)

chat = ChatHuggingFace(llm=llm)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/jovyan/.cache/huggingface/token
Login successful


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [4]:
class Origin(BaseModel):
    etymology: str = Field(description="Provide historical context, cultural references, or etymological information")
    anecdotes: str = Field(description="Include interesting stories, anecdotes, or theories about its origin")

class Idiom(BaseModel):
    Meaning: str = Field(description="Briefly explain its figurative meaning")
    context: str =  Field(description="Describe how it's used in conversation or writing")
    Origin: list[Origin]


idiom_query = "Break a leg"

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Idiom)

print(parser.get_format_instructions())

prompt = PromptTemplate(
    template="For a given idiom or proverb:\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

parser.get_format_instructions()
chain = prompt | chat | parser

chain.invoke({"query": idiom_query})

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"Meaning": {"title": "Meaning", "description": "Briefly explain its figurative meaning", "type": "string"}, "context": {"title": "Context", "description": "Describe how it's used in conversation or writing", "type": "string"}, "Origin": {"title": "Origin", "type": "array", "items": {"$ref": "#/definitions/Origin"}}}, "required": ["Meaning", "context", "Origin"], "definitions": {"Origin": {"title": "Origin", "type": "object", "properties": {"etymology": {"title": "Etymology", "description": "Provide historical context, cultural 

{'properties': {'Meaning': {'title': 'Meaning',
   'description': 'Break a leg is a phrase used to wish someone good luck, especially in the context of a performance or an audition.',
   'type': 'string'},
  'context': {'title': 'Context',
   'description': 'Break a leg is often used in the context of theater, where it is believed to have originated. It is also used in other performance settings, such as music, dance, or public speaking.',
   'type': 'string'},
  'Origin': {'title': 'Origin',
   'type': 'array',
   'items': [{'$ref': '#/definitions/Origin'}]}},
 'required': ['Meaning', 'context', 'Origin'],
 'definitions': {'Origin': {'title': 'Origin',
   'type': 'object',
   'properties': {'etymology': {'title': 'Etymology',
     'description': 'The phrase break a leg is thought to have originated in the theater, where it was used as a way to wish actors good luck before a performance. The phrase is believed to have been used as early as the 19th century.',
     'type': 'string'},
  